# `fsspec-reference-maker` tutorial

Created June 8th, 2020 by [Lucas Sterzinger](mailto:lsterzinger@ucdavis.edu) ([Twitter](https://twitter.com/lucassterzinger)) as part of the NCAR [Summer Internship in Parallel Computational Science (SIParCS)](https://www2.cisl.ucar.edu/siparcs)

If any part of this tutorial is now out of date, please feel free to open a pull request with a fix

In [ ]:
from fsspec_reference_maker.hdf import SingleHdf5ToZarr 
from fsspec_reference_maker.combine import MultiZarrToZarr

## Create metadata JSONs

### This function returns a list of S3 files for a given satellite, year, and day

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import zipfile
import logging
import fsspec
import json
from tqdm import tqdm
from glob import glob

In [ ]:
def get_file_list(sat,lyr,idyjl):
    # arguments
    # sat   goes-east,goes-west,himawari
    # lyr   year
    # idyjl day of year
    
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!

    #create strings for the year and julian day
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
    #use glob to list all the files in the directory
    if sat=='goes-east':
        file_location,var = fs.glob('s3://noaa-goes16/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    if sat=='goes-west':
        file_location,var = fs.glob('s3://noaa-goes17/ABI-L2-SSTF/'+syr+'/'+sjdy+'/*/*.nc'),'SST'
    
    return file_location

In [ ]:
flist = get_file_list("goes-east", 2020, 210)
urls = ["s3://" + f for f in flist]

### This function creates JSON metadata files for each of the S3 files in the local `jsons/` directory

These files point to the S3 location of the netCDF files, and only need to be created once. Tihs process took me about 10 minutes to generate the JSONs for 24 files. This function could easily be made to run in parallel for faster performance

In [ ]:
def gen_json(urllist):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False, default_cache_type="none"
    )
    zf = zipfile.ZipFile("out.zip", mode="w")
    for u in tqdm(urllist):
        with fsspec.open(u, **so) as inf:
            h5chunks = SingleHdf5ToZarr(inf, u, xarray=True, inline_threshold=100)
            with zf.open(os.path.basename(u) + ".json", 'w') as outf:
                outf.write(json.dumps(h5chunks.translate()).encode())

    # for u in tqdm(urllist):
    #     with fsspec.open(u, **so) as f:
    #         h5chunks = fsshdf.SingleHdf5ToZarr(f, u, xarray=True)
    #         j = h5chunks.translate()
    #         fname = u.split("/")[7]
    #         with open(f"./jsons/{fname}.json", "w") as fout:
    #             fout.write(json.dumps(j))


In [ ]:
gen_json(urls)

***
## Read remote netCDF files with xarray and fsspec

### First, create a list of JSON files

In [ ]:
json_list = sorted(glob("./jsons/*.json"))

### Then, loop over the files and use `fsspec.get_mapper()` to create mappers for each file object, creating a list of mappers

In [ ]:
mzz = MultiZarrToZarr(
    # "./jsons/*.json",
    "zip://*.json::out.zip",
    remote_protocol="s3",
    remote_options={'anon':True},
    xarray_kwargs={
        "decode_cf" : False,
        "mask_and_scale" : False,
        "decode_times" : False,
        "decode_timedelta" : False,
        "use_cftime" : False,
        "decode_coords" : False
    },
    with_mf='t'
)

In [ ]:
%%time
mzz.translate("combined.json")

%%time
fs = fsspec.filesystem("reference", fo="./combined.json", remote_protocol="s3", 
                        remote_options={"anon":True}, skip_instance_cache=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine='zarr')

In [ ]:
ds

In [ ]:
import hvplot.xarray

In [ ]:
ds.SST.where(ds.DQF==0).isel(t=0).hvplot.image(x='x', y='y', rasterize=True, aspect='equal', cmap='turbo')

### Take a subset of the data (in this case, the Gulf Stream)

### Select a single time with `.isel(t=14)`

In [ ]:
%%time
subset = ds.sel(x=slice(-0.01,0.07215601),y=slice(0.12,0.09))  #reduce to GS region

masked = subset.SST.where(subset.DQF==0)

masked.isel(t=14).plot(vmin=14+273.15,vmax=30+273.15,cmap='inferno')

### Plot a mean along the time axis (1-day average)

In [ ]:
%%time
subset = ds.sel(x=slice(-0.01,0.07215601),y=slice(0.12,0.09))  #reduce to GS region

masked = subset.SST.where(subset.DQF==0)

masked.mean("t", skipna=True).plot(vmin=14+273.15,vmax=30+273.15,cmap='inferno')